In [29]:
from miditok import REMIPlus, REMI
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile
from pathlib import Path

tokenizer = REMI()
midi = MidiFile('/home/mm/midi/maestro/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi')

tokens = tokenizer(midi)

tokens[0].tokens

['Bar_None',
 'Position_16',
 'Pitch_67',
 'Velocity_51',
 'Duration_1.5.8',
 'Position_29',
 'Pitch_72',
 'Velocity_67',
 'Duration_0.1.8',
 'Bar_None',
 'Position_2',
 'Pitch_71',
 'Velocity_43',
 'Duration_4.1.4',
 'Pitch_78',
 'Velocity_63',
 'Duration_3.0.8',
 'Position_3',
 'Pitch_61',
 'Velocity_39',
 'Duration_2.1.8',
 'Pitch_67',
 'Velocity_39',
 'Duration_2.0.8',
 'Bar_None',
 'Position_2',
 'Pitch_79',
 'Velocity_59',
 'Duration_0.5.8',
 'Position_12',
 'Pitch_60',
 'Velocity_47',
 'Duration_1.3.8',
 'Pitch_67',
 'Velocity_55',
 'Duration_1.1.8',
 'Pitch_70',
 'Velocity_63',
 'Duration_1.0.8',
 'Pitch_79',
 'Velocity_67',
 'Duration_0.6.8',
 'Position_19',
 'Pitch_75',
 'Velocity_71',
 'Duration_0.2.8',
 'Position_25',
 'Pitch_59',
 'Velocity_55',
 'Duration_1.5.8',
 'Pitch_67',
 'Velocity_59',
 'Duration_0.7.8',
 'Pitch_69',
 'Velocity_59',
 'Duration_0.5.8',
 'Pitch_75',
 'Velocity_79',
 'Duration_0.6.8',
 'Position_29',
 'Pitch_71',
 'Velocity_67',
 'Duration_0.2.8',
 'Ba

In [ ]:

# Constructs the vocabulary with BPE, from the tokenized files
tokenizer.learn_bpe(
    vocab_size=500,
    tokens_paths=list(Path("/home/mm/midi/maestro_tokens/").glob("**/*.json")),
    start_from_empty_voc=False,
)

# Saving our tokenizer, to retrieve it back later with the load_params method
tokenizer.save_params(Path("../data/tokenizer_params.json"))

In [ ]:
!ls /home/mm/midi/maestro_tokens/